# CNN beats NN beats kNN, 99.5% vs. 98.5% vs. 97%
Here's my second and third attempt classifying the MNIST handwritten digits. My first attempt used kNN and scored 97%. Next, I coded a fully connected feedforward neural network in C which scores 98.5%. Data augmentation pushes that to 99%. Third I added convolutions to my C program and score 99.5%!! 

UPDATE: Using TensorFlow (and GPUs) I built an ensemble of CNNs and scored an amazing 99.75% accuracy!! Check it out, [here][1].
  
UPDATE: I created a webpage where you can draw digits with your mouse and watch this network classify them, [here][2].

## The Training Data
Here are the first 54 training images. Kaggle's MNIST training dataset contains 42,000 images of size 28 x 28 grayscale pixels. Kaggle's MNIST test dataset contains 28,000 unlabeled images. Our model must learn to recognize digits from the training images and then predict the correct labels for the test images.

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/digits54.png)

[1]:https://www.kaggle.com/cdeotte/25-million-images-0-99757-mnist
[2]:http://www.ccom.ucsd.edu/~cdeotte/programs/MNIST.html

# NN Architecture - 98.5%
The images have 784 features ( = 28 by 28 pixels each with grayscale color from 0.0 to 1.0). This input feeds into a hidden layer of 1000 neurons which feeds into another hidden layer of 1000 neurons. The output layer has 10 neurons. The architecture is 784-1000-1000-10.  

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/MNISTnet2.jpeg)

# Initialization
For the hidden layer activations, I use ReLU. The output layer uses softmax regression with cross entropy cost. The network learns with stochastic back propagation gradient descent and the weights are initialized with Xavier-He initialization. Below is my C code: (also on GitHub [here][1])

[1]:https://github.com/cdeotte/MNIST-CNN-99.5

```
int layerSizes[10] = {0,0,0,0,0,0,784,1000,1000,10};
float* layers[10] = {0};
float* errors[10] = {0};
float*  weights[10] = {0};
// INITIALIZATION
void initNet(){
    // ALOCATE MEMORY
    layers[0] = (float*)malloc((layerSizes[0]+1) * sizeof(float));
    errors[0] = (float*)malloc(layerSizes[0] * sizeof(float));
    for (i=1;i<10;i++){
        layers[i] = (float*)malloc((layerSizes[i]+1) * sizeof(float));
        errors[i] = (float*)malloc(layerSizes[i] * sizeof(float));
        weights[i] = (float*)malloc(layerSizes[i] * (layerSizes[i-1]+1) * sizeof(float));
    }
    // RANDOMIZE WEIGHTS AND BIAS
    float scale;
    for (i=0;i<10;i++) layers[i][layerSizes[i]]=1.0;
    for (j=1;j<10;j++){
        // XAVIER-HE INITIALIZATION
        scale = 2.0 * sqrt(6.0/(layerSizes[j] + layerSizes[j-1]));
        if (j!=9 && activation==1) scale = scale * 1.41; // RELU
        else if (j!=9) scale = scale * 4.0; // TANH
        for (i=0;i<layerSizes[j] * (layerSizes[j-1]+1);i++)
            weights[j][i] = scale * ( (float)rand()/RAND_MAX - 0.5 );
        for (i=0;i<layerSizes[j];i++) // BIASES
            weights[j][(layerSizes[j-1]+1)*(i+1)-1] = 0.0;
    }
}
```

# Forward Propagation

```
int activation = 1; //ReLU
// FORWARD PROPAGATION
int forwardProp(int x){
    int i,j,k,imax;
    float sum, esum, max;
    // INPUT LAYER - RECEIVES 28X28 IMAGES
    for (i=0;i<784;i++) layers[10-numLayers][i] = trainImages[x][i];
    // HIDDEN LAYERS - RELU ACTIVATION
    for (k=11-numLayers;k<9;k++)
    for (i=0;i<layerSizes[k];i++){
        sum = 0.0;
        for (j=0;j<layerSizes[k-1]+1;j++)
            sum += layers[k-1][j]*weights[k][i*(layerSizes[k-1]+1)+j];
        if (activation==1) layers[k][i] = ReLU(sum);
        else if (activation==2) layers[k][i] = TanH(sum);
    }
    // OUTPUT LAYER - SOFTMAX ACTIVATION
    esum = 0.0;
    for (i=0;i<layerSizes[9];i++){
        sum = 0.0;
        for (j=0;j<layerSizes[8]+1;j++)
            sum += layers[8][j]*weights[9][i*(layerSizes[8]+1)+j];
        if (sum>30) return -1; //GRADIENT EXPLODED
        layers[9][i] = exp(sum);
        esum += layers[9][i];
    }
    // SOFTMAX FUNCTION
    max = layers[9][0]; imax=0;
    for (i=0;i<layerSizes[9];i++){
        if (layers[9][i]>max){
            max = layers[9][i];
            imax = i;
        }
        layers[9][i] = layers[9][i] / esum;
    }
    return imax;
}
```

# Backpropagation

```
float learn = 0.01;
float decay = 0.95;
// BACKPROPAGATION
int backProp(int x, int epoch, float *ent){
    int i, j, k, r = 0;
    float der=1.0;
    // FORWARD PROP FIRST
    int p = forwardProp(x);
    if (p==-1) return -1; // GRADIENT EXPLODED
    // CORRECT PREDICTION?
    int y = trainDigits[x];
    if (p==y) r=1;
    // OUTPUT LAYER - CALCULATE ERRORS
    for (i=0;i<layerSizes[9];i++){
        errors[9][i] = learn * (0-layers[9][i]) * pow(decay,epoch);
        if (i==y) {
            errors[9][i] = learn * (1-layers[9][i]) * pow(decay,epoch);
            if (layers[9][i]==0) return -1; // GRADIENT VANISHED
            *ent = -log(layers[9][i]);
        }
    }
    // HIDDEN LAYERS - CALCULATE ERRORS
    for (k=8;k>10-numLayers;k--)
    for (i=0;i<layerSizes[k];i++){
        errors[k][i] = 0;
        if (activation==2) der = (layers[k][i]+1)*(1-layers[k][i]); // TanH DERIVATIVE
        if (layers[k][i]>0 || activation==2) // ReLU DERIVATIVE
        for (j=0;j<layerSizes[k+1];j++)
            errors[k][i] += errors[k+1][j]*weights[k+1][j*(layerSizes[k]+1)+i]*der;
    }
    // UPDATE WEIGHTS - GRADIENT DESCENT
    for (k=11-numLayers;k<10;k++)
    for (i=0;i<layerSizes[k];i++)
        for (j=0;j<layerSizes[k-1]+1;j++)
            weights[k][i*(layerSizes[k-1]+1)+j] += errors[k][i]*layers[k-1][j];
    return r;
}
```

# Training, Validation, Prediction

First Kaggle's training set of 42000 images is split into a training subset of 31500 images = 75% x 42,000 and a validation subset of 10500 images = 25% x 42000 images. Each epoch, every training subset image is fed forward into the network and the weights are adjusted by back propagating errors. The learning rate begins at 0.01 and decreases each epoch by 0.95. The batch size is 1 and the order the images are fed into the network is randomized each epoch. By monitoring validation accuracy, we can tune the network's architecture and regularization. Lastly, we classify Kaggle's test set of 28,000 images with the network and submit online. With only dropout (25%), this network achieves 98.25% accuracy. With dropout and data augmentation, this network reaches 99%.


# Regularization
To prevent overfitting and to improve generalization, a combination of dropout and data augmentation are employed. Data augmentation is the technique of creating new training images by randomly distorting the ones you have. My program randomly shifts, scales, and rotates the training data. Below is an illustration. The images on the left are the first 24 original MNIST digits. The images on the right have been augmented.

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/MNISTaugment2.png)

# CNN Architecture - 99.5%
It's amazing how much more accurate CNNs are over fully connected NNs. By adding convolutions to my C program, I was able to build and train a CNN network. Based on LeNet5's architecture (picture below) I created the following nine layer network 784-10C3-10C3-P2-20C3-20C3-P2-128-10 (not pictured).  

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/LeNet5.png)

# Training and Validation  

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/MNISTconfusion3.png)
![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/MNISTiterations2.png)

# Errors
Looking at the confusion matrix above, you see that the network had the most difficulty distinquishing fours and nines. The six digits below to the left are fours misclassified as nines. And the six on the right are nines misclassified as fours.
  
![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/4and9.png)
  
None-the-less, this CNN achieves a training accuracy of 99.25% and a validation accuracy of 99.5%. Next we classify Kaggle's test images and submit.

# Prediction
![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/MNISTresult2.png)

# Conclusion
It was fun to watch models progressively achieve better accuracy classifying MNIST's handwritten digits. And programming everything in C from scratch taught me a lot. First kNN achieved 97%, next a fully connected net achieved 98.5%. Adding data augmentation increased the fully connected net's accuracy to 99%. And finally I saw a CNN achieve 99.5%. That was impressive! I uploaded my C code to GitHub [here][3]. 
  
UPDATE: Using TensorFlow (and GPUs) I built an ensemble of CNNs and scored an amazing 99.75% accuracy!! Check it out, [here][1].  
  
UPDATE: I created a webpage where you can draw digits with your mouse and watch this network classify them, [here][2]:

[1]:https://www.kaggle.com/cdeotte/25-million-images-0-99757-mnist
[2]:http://www.ccom.ucsd.edu/~cdeotte/programs/MNIST.html
[3]:https://github.com/cdeotte/MNIST-CNN-99.5

In [1]:
cat("Thanks for reading my kernel. I look forward to comments and questions")

Thanks for reading my kernel. I look forward to comments and questions

# How does a CNN see MNIST?
LeNet5 uses the architecture 784-6C5-P2-16C5-P2-120-84-10. A layer of 6 convolution filters of size 5x5 is the first layer to process an incoming image. These filters only see 5x5 regions in the image and each filter looks for a specific pattern and sends a strong activation when found. Let's call these six filters: red, orange, yellow, green, blue, and purple. Below are examples of patterns that four of these six filters (from one particular trained network) look for.  

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/filter1.png)
![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/filter2.png)
  
Notice how the yellow filter looks for a diagonal line. The orange filter looks for a left vertical edge. The blue filter looks for a bottom right diagonal edge. And the purple filter looks for a horizontal line.

The output from these fiters are fed into a second layer of 16 convolution filters (after passing through a max pooling layer). Each of these 16 filters can only see a 14x14 region in the image. These filters look for certain combinations of the features discovered by the first set of filters. Below are examples of patterns that four of these sixteen filters (from one particular trained network) look for.  

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/filter3.png)
![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/filter4.png)
  
Notice how the first filter (top left) looks for a curved diagonal line. The second filter (top right) looks for a small loop in the bottom of the range with a line in the top of the range. The third filter (bottom left) looks for a small loop in the bottom of the range with a vertical line up the left side. And the fourth filter (bottom right) looks for a pair of parallel horizontal lines.  
  
All these filters search for specific things and then they report their findings to the network's final dense layers which classfiy the digit.

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/layers.jpeg)
![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/2020/filters.jpeg)